In [10]:
from matplotlib import pyplot
from keras.datasets import cifar10
import numpy as np 
import pandas as pd

In [11]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [7]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [12]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]* X_train.shape[2]* X_train.shape[3])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2] * X_test.shape[3])
y_train = y_train.reshape(y_train.shape[0], )
y_test = y_test.reshape(y_test.shape[0], )

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(50000, 3072)
(10000, 3072)
(50000,)
(10000,)
